# Import dependencies

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from sqlalchemy import create_engine

# Import datasets

In [2]:
# Reading the unemployment csv file and creating a dataframe
unemployment_csv_file = "input/unemployment_rate.csv"
unemployment_df = pd.read_csv(unemployment_csv_file, encoding ='cp1252')

unemployment_df.head()

,Series ID,Region Name,State,2000,2001,2002,2003,2004,2005,2006,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,SCABBE1URN,Abbeville County,SC,3.8,6.6,7.7,9.1,8.2,7.7,8.8,...,14.2,13.6,12.5,10.6,9.5,7.7,6.9,5.5,4.6,4.2
1,LAACAD0URN,Acadia Parish,LA,5.8,6.0,6.3,6.8,5.8,6.1,3.6,...,6.5,7.3,6.7,6.0,6.0,5.8,6.9,7.4,5.9,5.4
2,VAACCO1URN,Accomack County,VA,3.0,3.4,4.2,4.4,4.7,4.7,4.3,...,6.5,7.9,8.4,7.9,7.2,6.7,5.5,5.0,4.9,4.1
3,IDADAC1URN,Ada County,ID,3.4,3.9,4.7,4.9,4.1,3.3,2.7,...,8.9,8.3,7.3,6.2,5.2,4.2,3.7,3.3,2.8,2.4
4,OKADAI1URN,Adair County,OK,3.5,4.1,4.8,7.8,6.2,4.8,4.4,...,8.0,10.5,9.2,9.1,9.8,8.0,6.1,5.9,5.0,4.4


In [3]:
# Reading the income csv file and creating a dataframe
income_csv_file ="input/income_data.csv"
income_df = pd.read_csv(income_csv_file, encoding = "UTF-8")

income_df.head()

,id,State_Code,State_Name,State_ab,County,City,Place,Type,Primary,Zip_Code,Area_Code,ALand,AWater,Lat,Lon,Mean,Median,Stdev,sum_w
0,1011000,1,Alabama,AL,Mobile County,Chickasaw,Chickasaw city,City,place,36611,251,10894952,909156,30.771450,-88.079697,38773,30506,33101,1638.260513
1,1011010,1,Alabama,AL,Barbour County,Louisville,Clio city,City,place,36048,334,26070325,23254,31.708516,-85.611039,37725,19528,43789,258.017685
2,1011020,1,Alabama,AL,Shelby County,Columbiana,Columbiana city,City,place,35051,205,44835274,261034,33.191452,-86.615618,54606,31930,57348,926.031000
3,1011030,1,Alabama,AL,Mobile County,Satsuma,Creola city,City,place,36572,251,36878729,2374530,30.874343,-88.009442,63919,52814,47707,378.114619
4,1011040,1,Alabama,AL,Mobile County,Dauphin Island,Dauphin Island,Town,place,36528,251,16204185,413605152,30.250913,-88.171268,77948,67225,54270,282.320328


In [4]:
# Reading the auto insurance csv file and creating a dataframe
auto_ins_csv_file = "input/new_auto_ins_data.csv"
auto_ins_df = pd.read_csv(auto_ins_csv_file)

auto_ins_df.head()

,Unnamed: 0,rank,state,full_coverage,minimum_coverage,difference,State_Name,State_ab
0,0,1,Michigan,8723,5282,3441,Michigan,MI
1,1,2,Rhode Island,3847,1589,2258,Rhode Island,RI
2,2,3,Louisiana,3525,1329,2196,Louisiana,LA
3,3,4,Kentucky,3418,1338,2079,Kentucky,KY
4,4,5,Florida,3370,2565,805,Florida,FL


# Create dataframes

In [5]:
## Using the unemployment csv we only want to keep the relevant columns 
unemployment_df = unemployment_df[["State", "2018" ]]

## Convert "State"column values to strings
unemployment_df["State"] = unemployment_df["State"].astype(str)

## The dataset produced one row per county therefore to calculate the state average we need to group by state
clean_unemployment = round(unemployment_df.groupby("State").mean(),2)
clean_unemployment = clean_unemployment.reset_index()

## Rename columns
clean_unemployment.columns = ["State", "Unemployment Rate"]

## Remove all the whitespace in the column "State"
clean_unemployment["State"] = clean_unemployment["State"].str.strip()

clean_unemployment.head()

,State,Unemployment Rate
0,AK,8.53
1,AL,4.53
2,AR,4.24
3,AZ,6.65
4,CA,5.19


In [6]:
## Using the income csv we only want to keep the relevant columns
income_df = income_df[["State_ab", "Mean" ]]

## Convert "State_ab" column values to strings
income_df["State_ab"] = income_df["State_ab"].astype(str)

## The dataset produced one row per zip code therefore to calculate the state average we need to group by state
clean_income = round(income_df.groupby("State_ab").mean(),2)
clean_income = clean_income.reset_index()

## Rename columns
clean_income.columns = ["State", "Income"]

## Remove all whitespace in the column "State"
clean_income["State"] = clean_income["State"].str.strip()

clean_income.head()

,State,Income
0,AK,77670.21
1,AL,53612.93
2,AR,52060.36
3,AZ,62578.07
4,CA,78126.74


In [10]:
## Using the income csv we only want to keep the relevant columns
clean_auto_ins = auto_ins_df[["State_ab", "full_coverage"]]

## Convert "State_ab" column values to strings
clean_auto_ins["State_ab"] = clean_auto_ins["State_ab"].astype(str)

## Rename columns
clean_auto_ins.columns = ["State", "Auto Ins. Rate"]

## Remove all whitespace in the column "State"
clean_auto_ins["State"] = clean_auto_ins["State"].str.strip()

clean_auto_ins.head()

,State,Auto Ins. Rate
0,MI,8723
1,RI,3847
2,LA,3525
3,KY,3418
4,FL,3370


# Merge data

In [11]:
## Merge all 3 clean data sets using state as the primary key

merge_df = clean_unemployment.merge(clean_income, on = "State").merge(clean_auto_ins, on = "State")
merge_df

,State,Unemployment Rate,Income,Auto Ins. Rate
0,AK,8.53,77670.21,1502
1,AL,4.53,53612.93,2078
2,AR,4.24,52060.36,2213
3,AZ,6.65,62578.07,2699
4,CA,5.19,78126.74,1804
5,CO,3.17,73322.82,3164
6,CT,4.10,89227.22,2619
7,DE,4.03,70142.34,2513
8,FL,3.75,60887.89,3370
9,GA,4.36,60354.39,2619
